In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import statsmodels
import matplotlib.pyplot as plt
import seaborn
import dateutil
from datetime import datetime
import math
import plotly.graph_objects as go
import plotly.express as px
import math
import functions.functions as base_fx
import functions.portfolio_functions as portfolio_fx
import functions.algorithms as algos
from statsmodels.tsa.stattools import coint

import importlib


In [8]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)


<module 'functions.algorithms' from '../functions/algorithms.py'>

In [2]:
symbols = ['0700.HK','NVDA.US','SNE.US','NTDOY.US','AMD.US','ATVI.US','EA.US','ZNGA.US','TTWO.US','GME.US','MAT.US','HAS.US','GAW.LSE','AKAM.US','UBI.PA']


# Get Data

In [4]:
quotes = pd.DataFrame()
#start = '2018-01-1'
start = '2016-01-1'
end = '2020-05-25'

In [5]:
for s in symbols:
    print(s)
    endpoint = f'https://eodhistoricaldata.com/api/eod/{s}?from={start}&to={end}&api_token={API_KEY}&interval=d&fmt=csv'
    df = pd.read_csv(endpoint)
    #print(df.head(1))
    df = df[df['Adjusted_close'] > 0]
    #print(df.head(1))
    if len(quotes) == 0:
        quotes['Date'] = df['Date']
        quotes[s] = df['Adjusted_close']
    else:
        new_df = pd.DataFrame()
        new_df['Date'] = df['Date']
        new_df[s] = df['Adjusted_close']
        quotes = quotes.merge(new_df, how="inner", on="Date")
   

0700.HK
NVDA.US
SNE.US
NTDOY.US
AMD.US
ATVI.US
EA.US
ZNGA.US
TTWO.US
GME.US
MAT.US
HAS.US
GAW.LSE
AKAM.US
UBI.PA


In [154]:
# backup quotes
old_quotes = quotes

In [155]:
quotes = quotes.dropna()
quotes = quotes.reset_index()
quotes['Date'] = pd.to_datetime(quotes['Date'])
quotes = quotes.set_index('Date')


# Params

In [156]:
symbol1 = 'LUV.US'
symbol2 = 'GD.US'

In [157]:
# Indicators
macd_fast_points = 1
macd_slow_points = 2
max_leverage = 2

# Could be changed
mavg_1 = 5 # default 5
mavg_2 = 60 #default 60

## Optimize trading limits looking at zscore
limit = 0.75 #default 1
exit_limit = 0.5 #default -.75
leverage_limit = 1
zscore_limit = 1.5

## Optimize these with bruteforce
vol_interval = 30
vol_peak_period = 365 # 1 year
vol_mean_peak_period = 60 # 2 months
coint_limit =  1
coint_period = 365


In [158]:
c = coint(quotes[symbol1], quotes[symbol2])
print(c[1])

0.04441327343737045


# Trade

In [159]:
settings = {
    'symbol1': symbol1,
    'symbol2': symbol2,
    'mavg_1' : mavg_1,
    'mavg_2': mavg_2,
    'macd_fast_points': macd_fast_points,
    'macd_slow_points': macd_slow_points,
    'vol_interval': vol_interval,
    'vol_peak_period': vol_peak_period,
    'vol_mean_peak_period': vol_mean_peak_period,
    'limit': limit,
    'leverage_limit': leverage_limit,
    'exit_limit': exit_limit,
    'max_leverage': max_leverage,
    'coint_limit': coint_limit,
    'coint_period': coint_period,
    'sl_limit': -0.05
}

In [160]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)
portfolio, mavg_zscore = algos.pair_trade_with_zscore_limit_and_corr(settings, quotes)

Return: 
56.15416298503111%
Vol:
0.17832245009330136


In [151]:
portfolio_fx.plot_trades_on_zscore_for_symbol(mavg_zscore, portfolio, symbol1)
portfolio_fx.plot_trades_on_zscore_for_symbol(mavg_zscore, portfolio, symbol2)


In [117]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)
#print(quotes)
portfolio_fx.plot_trades_on_series(symbol1, symbol2, quotes[symbol1], quotes[symbol2], portfolio)


    symbol  type       amount  bought_at  bought_on status  leverage  \
0   LUV.US  SELL  5000.000000    43.1924 2015-03-23  CLOSE       2.0   
1    GD.US   BUY  5000.000000   121.9252 2015-03-23  CLOSE       2.0   
2   LUV.US   BUY  5154.637254    40.6453 2015-04-13  CLOSE       1.0   
3    GD.US  SELL  5154.637254   121.5505 2015-04-13  CLOSE       1.0   
4   LUV.US   BUY  5180.310530    38.8579 2015-05-06  CLOSE       2.0   
5    GD.US  SELL  5180.310530   124.1298 2015-05-06  CLOSE       2.0   
6   LUV.US   BUY  4625.371252    32.3063 2015-07-15  CLOSE       1.0   
7    GD.US  SELL  4625.371252   133.4553 2015-07-15  CLOSE       1.0   
8   LUV.US  SELL  4838.383550    35.5055 2015-08-24  CLOSE       2.0   
9    GD.US   BUY  4838.383550   127.6187 2015-08-24  CLOSE       2.0   
10  LUV.US  SELL  4698.697432    39.1600 2015-10-21  CLOSE       2.0   
11   GD.US   BUY  4698.697432   130.3246 2015-10-21  CLOSE       2.0   
12  LUV.US   BUY  4389.380254    35.8842 2016-02-04  CLOSE      